In [175]:
from pulp import *

In [176]:
# Lsit (TimePeriods)
t = [0,1,2,3,4,5,6]

In [177]:
# Parameters and Data
demand = {1:100, 2:100, 3:150, 4:200, 5:150, 6:100} # Demand data
UPC = {1:7, 2:8, 3:8, 4:8, 5:7, 6:8} # Unit Production Cost (Excluding Labor)
UHC = {1:3, 2:4, 3:4, 4:4, 5:3, 6:2} # Unit Holding Cost
URLC = {1:15, 2:15, 3:18, 4:18, 5:15, 6:15} # Unit Regular Labor Cost
UOLC = {1:22.5, 2:22.5, 3:27, 4:27, 5:22.5, 6:22.5} # Unit Overtime Labor Cost
R_MH = {1:120, 2:130, 3:120, 4:150, 5:100, 6:100} # Available Man-hours R (Regular time) Labor
O_MH = {1:30, 2:40, 3:40, 4:30, 5:30, 6:30} # Available Man-hours O (Overtime) Labor

In [178]:
# Setting the Problem
prob = LpProblem("Aggregate Production Planning: Fixed Work Force Model", LpMinimize)

In [179]:
# Desicion Variables
Xt = LpVariable.dicts("Quantity Produced", t, 0)
It = LpVariable.dicts("Inventory", t, 0)
Rt = LpVariable.dicts("R_Labor Used", t, 0)
Ot = LpVariable.dicts("O_Labor Used", t, 0)

In [180]:
# Objective Function
prob += lpSum(UPC[i]*Xt[i] for i in t[1:])+lpSum(UHC[i]*It[i] for i in t[1:])+lpSum(URLC[i]*Rt[i] for i in t[1:])+lpSum(UOLC[i]*Ot[i] for i in t[1:])

In [181]:
# Constraints
It[0] = 3
for i in t[1:]:
    prob += (Xt[i] + It[i-1]- It[i]) == demand[i] # Inventory-Balancing Constraints
for i in t[1:]:
    prob += Xt[i] - Rt[i] - Ot[i] == 0  # Time Required to produce products
for i in t[1:]:
    prob += Rt[i] <= R_MH[i] # Regular Time Required
for i in t[1:]:
    prob += Ot[i] <= O_MH[i] # Over Time Required

In [182]:
prob.solve()
print("Solution Status = ", LpStatus[prob.status])

Solution Status =  Optimal


In [183]:
# Print the solution of the Decision Variables
for v in prob.variables():
    if v.varValue>0:
        print(v.name, "=", v.varValue)

Inventory_1 = 23.0
Inventory_2 = 70.0
Inventory_3 = 40.0
Inventory_4 = 20.0
O_Labor_Used_2 = 17.0
O_Labor_Used_4 = 30.0
O_Labor_Used_5 = 30.0
Quantity_Produced_1 = 120.0
Quantity_Produced_2 = 147.0
Quantity_Produced_3 = 120.0
Quantity_Produced_4 = 180.0
Quantity_Produced_5 = 130.0
Quantity_Produced_6 = 100.0
R_Labor_Used_1 = 120.0
R_Labor_Used_2 = 130.0
R_Labor_Used_3 = 120.0
R_Labor_Used_4 = 150.0
R_Labor_Used_5 = 100.0
R_Labor_Used_6 = 100.0


In [184]:
# Print Optimal
print("Total Production Plan Cost = ", value(prob.objective))

Total Production Plan Cost =  20192.5


In [96]:
for i in t[1:]:
    print((Xt[i] - Rt[i] - Ot[i]) == 0) # Time Required to produce products


-O_Labor_Used_1 + Quantity_Produced_1 - R_Labor_Used_1 = 0
-O_Labor_Used_2 + Quantity_Produced_2 - R_Labor_Used_2 = 0
-O_Labor_Used_3 + Quantity_Produced_3 - R_Labor_Used_3 = 0
-O_Labor_Used_4 + Quantity_Produced_4 - R_Labor_Used_4 = 0
-O_Labor_Used_5 + Quantity_Produced_5 - R_Labor_Used_5 = 0
-O_Labor_Used_6 + Quantity_Produced_6 - R_Labor_Used_6 = 0


In [97]:
for i in t[1:]:
    print(Rt[i] <= R_MH[i]) # Regular Time Required


R_Labor_Used_1 <= 120
R_Labor_Used_2 <= 130
R_Labor_Used_3 <= 120
R_Labor_Used_4 <= 150
R_Labor_Used_5 <= 100
R_Labor_Used_6 <= 100
